In [1]:
import pandas as pd
# import numpy as np

In [2]:
database_type = "/full"

The `patient` table includes general information about the patient admissions (for example, demographics, admission and discharge details). 
See: http://eicu-crd.mit.edu/eicutables/patient/

In [27]:
# loads patient table
patients = pd.read_csv('../eICU' + database_type + '/patient.csv', nrows=999999) 
patients = patients[['patientunitstayid', 'admissionweight']].set_index('patientunitstayid')
patients.head()

,admissionweight
patientunitstayid,
141168,84.3
141178,54.4
141179,NaN
141194,73.9
141196,NaN


The `vitalperiodic` table comprises data that is consistently interfaced from bedside vital signs monitors into eCareManager. 
Data are generally interfaced as 1 minute averages, and archived into the `vitalperiodic` table as 5 minute median values. 
For more detail, see: http://eicu-crd.mit.edu/eicutables/vitalPeriodic/

In [12]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv', nrows=999999)
columns = ['patientunitstayid', 'observationoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic']
vitalperiodic = vitalperiodic[columns].set_index('observationoffset')
vitalperiodic = vitalperiodic.sort_values(by='observationoffset')
vitalperiodic.head()

,patientunitstayid,temperature,heartrate,respiration,systemicsystolic
observationoffset,,,,,
-1445,148349,NaN,60.0,20.0,NaN
-1440,148349,NaN,60.0,21.0,NaN
-1435,150049,NaN,96.0,19.0,NaN
-1435,146418,NaN,71.0,23.0,NaN
-1435,148349,NaN,60.0,21.0,NaN


The `vitalAperiodic` table provides invasive vital sign data that is recorded at irregular intervals. 
See: http://eicu-crd.mit.edu/eicutables/vitalAperiodic/

In [14]:
vitalaperiodic = pd.read_csv('../eICU' + database_type + '/vitalAperiodic.csv', nrows=999999)
columns = ['observationoffset', 'patientunitstayid','noninvasivesystolic','cardiacoutput','cardiacinput']
vitalaperiodic = vitalaperiodic[columns].set_index('observationoffset')
vitalaperiodic = vitalaperiodic.sort_values(by='observationoffset')
vitalaperiodic.head()

,patientunitstayid,noninvasivesystolic,cardiacoutput,cardiacinput
observationoffset,,,,
-4320,218289,187.0,NaN,NaN
-4319,218289,178.0,NaN,NaN
-3883,218287,138.0,NaN,NaN
-3659,218287,134.0,NaN,NaN
-3632,218287,147.0,NaN,NaN


The `lab` table provides Laboratory tests that have have been mapped to a standard set of measurements. 
Unmapped measurements are recorded in the customLab table.
See: http://eicu-crd.mit.edu/eicutables/lab/

In [19]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv', nrows=999999)
columns = ['labresultoffset', 'patientunitstayid','labtypeid','labname', 'labresult', 'labmeasurenamesystem', 'labmeasurenameinterface']
lab = lab[columns].set_index('labresultoffset')
lab = lab.sort_values(by='labresultoffset')
lab.head()

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [ ]:
# Pull out dataframe for labresults 'WBC x 1000', 'lactate', 'creatinine'


In [18]:
# converts all occurances of urinary creatinine in labname col to creatinine
# alternativy check labtypeid creatinine = 1,  urinary creatinine = 4

lab.loc[lab['labname'].str.contains("urinary creatinine"), 'labname'] = "creatinine"
lab.loc[(lab['labname'].str.contains("creatinine"))]

,patientunitstayid,labtypeid,labname,labresult,labmeasurenamesystem,labmeasurenameinterface
labresultoffset,,,,,,
1701,141168,4,creatinine,173.12,mg/dL,mg/dL
2026,141168,1,creatinine,2.95,mg/dL,mg/dL
1133,141168,1,creatinine,2.30,mg/dL,mg/dL
516,141168,1,creatinine,1.95,mg/dL,mg/dL
-280,141178,1,creatinine,0.70,mg/dL,mg/dL
...,...,...,...,...,...,...
10778,180812,1,creatinine,1.32,mg/dL,mg/dL
4837,180812,1,creatinine,1.25,mg/dL,mg/dL
13693,180812,1,creatinine,1.33,mg/dL,mg/dL


In [14]:
# showing all results for parient with id 154910

lab['patientunitstayid'] = lab['patientunitstayid'].astype(str)
lab.loc[(lab['patientunitstayid'].str.contains("154910"))]

In [14]:
intakeOutput = pd.read_csv('../eICU' + database_type + '/intakeOutput.csv', nrows=999999)
intakeOutput.head()

,intakeoutputid,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,dialysistotal,nettotal,intakeoutputentryoffset,cellpath,celllabel,cellvaluenumeric,cellvaluetext
0,9301638,205505,661,0.0,0.0,0.0,0.0,661,flowsheet|Flowsheet Cell Labels|I&O|Weight|Bod...,Bodyweight (lb),238.2,238.2
1,9301639,205505,661,0.0,0.0,0.0,0.0,661,flowsheet|Flowsheet Cell Labels|I&O|Weight|Bod...,Bodyweight (kg),108.0,108.0
2,9301642,150708,3523,727.0,0.0,0.0,727.0,3523,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,P.O.,200.0,200.0
3,9301643,150708,3523,727.0,0.0,0.0,727.0,3523,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (mL)-sodium chloride 0.9 % flush IVPB 5...,527.0,527.0
4,9301645,158159,24801,120.0,0.0,0.0,120.0,24801,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,P.O.,120.0,120.0


In [22]:
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutputUrine["patientunitstayid"]= intakeOutput["patientunitstayid"].astype(str) 
patientIntakeOutputUrine = intakeOutputUrine.loc[(intakeOutputUrine['patientunitstayid'].str.contains("178962"))]
patientIntakeOutputUrine.sort_values(by='intakeoutputoffset')

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,intakeoutputid,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,dialysistotal,nettotal,intakeoutputentryoffset,cellpath,celllabel,cellvaluenumeric,cellvaluetext
813689,16734129,178962,-4627,300.0,300.0,0.0,0.0,-4627,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,300.0,300.0
85278,10086913,178962,-4417,240.0,400.0,0.0,-160.0,-4417,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,400.0,400.0
294647,11992927,178962,-4237,0.0,300.0,0.0,-300.0,-4237,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,300.0,300.0
365555,12641418,178962,-4147,0.0,225.0,0.0,-225.0,-4147,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,225.0,225.0
510054,13960123,178962,-4044,0.0,125.0,0.0,-125.0,-4044,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,125.0,125.0
408163,13028278,178962,-3854,0.0,375.0,0.0,-375.0,-3854,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,375.0,375.0
495042,13822200,178962,-3205,300.0,200.0,0.0,100.0,-3205,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,200.0,200.0
525701,14104055,178962,-3037,0.0,250.0,0.0,-250.0,-3037,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,250.0,250.0
135627,10542811,178962,-2437,0.0,400.0,0.0,-400.0,-2437,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,400.0,400.0
644552,15188036,178962,-1963,0.0,400.0,0.0,-400.0,-1963,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Urine,400.0,400.0
